# Start and check Dbus Daemon

## Start Dbus daemon

In [ ]:
!eval `dbus-launch --sh-syntax`; echo "D-Bus per-session daemon address is: $DBUS_SESSION_BUS_ADDRESS"

##  Check Dbus session environment variable

In [2]:
import os
print("export DBUS_SESSION_BUS_ADDRESS=\'{}\'".format(os.environ['DBUS_SESSION_BUS_ADDRESS']))

export DBUS_SESSION_BUS_ADDRESS='unix:abstract=/tmp/dbus-JtFwD5OlpE,guid=7a0e63c56c622705c370e05b58f67c16'


## Set Dbus environment variable

In [ ]:
import os
out = !dbus-launch --sh-syntax
a = out[0].split('=', 1)
a0 = a[0]
a1 = "".join(a[1:])[:-1]
print out
print a0, a1
os.environ[a0] = a1
print a0, os.environ[a0]

# Start si446x

## Command line invocation

In [ ]:
%env DBUS_SESSION_BUS_ADDRESS='unix:abstract=/tmp/dbus-fmgWcS17m4,guid=de5bc5f9338a6eda64c0e5ae58ec6ea9'
!python -m si446x

## Python invocation

In [ ]:
from si446x              import reactor_loop
from twisted.python      import log
import sys

if __name__ == '__main__':
    log.startLogging(sys.stdout)
    reactor_loop()

# Start pydbus

In [1]:
from pydbus import SessionBus

#get the session bus
bus = SessionBus()
#get the object
radio = bus.get("org.tagnet.si446x", "/org/tagnet/si446x/0/0")

# Query Radio Driver Status

In [2]:
#call the methods and print the results
reply = radio.status()
print(reply)

ON, <States=S_RX_ON>, unshuts 0
 RX: packets 0, len_errors 0, timeouts 0, sync_errors 8, crc_errors 0, rssi 0, 
 TX: packets 0, errors 0, timeouts 0, power 32, 
 node P211, version 0.1.6 [2017.04.19 13:27:48.308264]


# Get Radio Device Info

### Silicon Labs Radio Part Info

The known device version information as published on Silicon Labs Message Board:

ROMID=3,REV=B1B
4060
4063
4355
4362
4438
4455
4460
4461
4463
4464

ROMID=6,REV=C2A
4055
4060
4063
4355
4362
4438
4455
4460
4461
4463
4464

ROMID=6,REV=A2A
4467
4468

(see: http://community.silabs.com/t5/Proprietary-Knowledge-Base/Using-PART-INFO-command-to-identify-EZRadio-PRO-part-number/ta-p/194617)

### Get Radio Part Info

In [27]:
from si446x.si446xdef import *

#create get part info radio command message
request = read_cmd_s.parse('\x00' * read_cmd_s.sizeof())
request.cmd='PART_INFO'
cmd = read_cmd_s.build(request)
#call the methods and print the results
rsp = radio.spi_send_recv(cmd, read_part_info_rsp_s.sizeof(),
                          read_cmd_s.name, read_part_info_rsp_s.name)    
if (rsp):
    response = read_part_info_rsp_s.parse(bytearray(rsp))
    print("Part Number: %x, rev: 0x%x, id: 0x%x, romid: 0x%x"%
          (response.part, response.chiprev, response.id, response.romid))
    print(rsp[1:])

request.cmd='FUNC_INFO'
cmd = read_cmd_s.build(request)
rsp = radio.spi_send_recv(cmd, read_func_info_rsp_s.sizeof(),
                          read_cmd_s.name, read_func_info_rsp_s.name)    
if (rsp):
    response = read_func_info_rsp_s.parse(bytearray(rsp))
    print("Firmware: %d.%d.%d, patch: 0x%x, func: 0x%x"%
          (response.revext, response.revbranch, response.revint, response.patch, response.func))
    print(rsp[1:])

Part Number: 4463, rev: 0x11, id: 0xf, romid: 0x3
[17, 68, 99, 0, 0, 15, 0, 3]
Firmware: 3.0.15, patch: 0x0, func: 0x1
[3, 0, 15, 0, 0, 1]


# Start Dbus and si446x Together

In [19]:
!eval `dbus-launch --sh-syntax python -m si446x`; echo "D-Bus per-session daemon address is: $DBUS_SESSION_BUS_ADDRESS"

^C


# Send packets as fast as possible

In [104]:
from time import sleep

buf = bytearray('\x00' * 250)
buf[0] = 250
pwr = 33
for i in range(1,50):
    buf[1] = i
    rsp = radio.send(buf, pwr)
    sleep(.5)
    print('.',)



.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
